# Tutorial about Sweeper

In this tutorial, you will learn how to:
- make a simple sweep
- load data from a sweep
- types of sweep instructions
- make a multi-dimensional sweep
- make a parallel sweep
- define initial and final configurations
- add custom pre/post processes
- add custom pre/post readout processes
- usability examples

Furthermore, this tutorial includes a breakdown of the Sweeper class for more complex measurements.

## Basic imports

In [1]:
import sys, os
import tempfile
import numpy as np
import matplotlib.pyplot as plt
from qcodes import Parameter, DelegateParameter, initialise_or_create_database_at, load_or_create_experiment, load_by_id

# Let's create a dummy qcodes database
db_path = os.path.join(tempfile.gettempdir(),'test.db')
initialise_or_create_database_at(db_path)
experiment = load_or_create_experiment(experiment_name='test',sample_name='no sample')

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\manip.batm\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\manip.batm\.qcodes\logs\221125-9476-qcodes.log


## Simple sweep

In [2]:
# Let's create some dummy parameters
from qcodes.validators import Numbers, Bool

x1 = Parameter('x1', unit='V', set_cmd=None, get_cmd=None, initial_value=0) 
x2 = Parameter('x2', unit='V', set_cmd=None, get_cmd=None, initial_value=0, vals=Numbers(-2, 2)) # with safety limits

y1 = Parameter('y1', unit='nA', set_cmd=None, get_cmd=lambda: 0) # always returns 0
y2 = Parameter('y2', unit='pA', set_cmd=None, get_cmd=np.random.random) # random number

In [3]:
# Imagine we want to sweep linearly x1 from 0 to -1V with 11pts, and repeat this 5 times. 
from qube import Sweeper
sw = Sweeper()

# we add sweep instructions
sw.sweep_linear(x1,0,-1,dim=1) # note that we don't need to define the number of points here
# Moreover, by convention, sweep dimensions starts with dim=1. 
# dim=0 is reserved for real-time axis such as AWG waveform ramps.

# we set the readouts
sw.set_readouts(y1,y2)

# we specify the number of points to sweep
sw.set_sweep_shape([11,5]) # [dim1 pts, dim2 pts, ...]
# Note that in dim=2 we don't have instructions. So, it will be simply a repetition.

# we set a waiting time before each readout to slow down this dummy sweep
sw.pre_readout_wait = 0.05 # in seconds

# we can execute the sweep
run_id = sw.execute() # it returns the run_id

Starting experimental run with id: 159. 
The measurement will take 0:00:03.337221


  0%|          | 0/55 [00:00<?, ?it/s]

## Load data from a sweep

In [4]:
# We can checkout the data in the qcodes database
ds = load_by_id(run_id)

# We can access for example to the x1 swept values
print(ds.get_parameter_data()['x1'])

{'x1': array([ 0. , -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1. ])}


In [5]:
# Or the readout values of y1 and y2
print(ds.get_parameter_data()['y1'])
print(ds.get_parameter_data()['y2'])

{'y1': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])}
{'y2': array([0.83308057, 0.3568504 , 0.89584079, 0.86529821, 0.38165725,
       0.98894172, 0.05323251, 0.96056645, 0.94723315, 0.01162523,
       0.38148303, 0.23438319, 0.08239313, 0.0685966 , 0.48470968,
       0.00134395, 0.92387966, 0.09662723, 0.62930332, 0.58631573,
       0.36103508, 0.33380163, 0.58129853, 0.92662235, 0.08525292,
       0.94870166, 0.004319  , 0.23109995, 0.1739997 , 0.2564684 ,
       0.73142759, 0.84241832, 0.53631401, 0.02083012, 0.75244789,
       0.3296243 , 0.27881912, 0.23633295, 0.07686284, 0.70451522,
       0.03538699, 0.43884228, 0.27231484, 0.31930112, 0.72320962,
       0.56226175, 0.25965653, 0.24299394, 0.55709103, 0.22072269,
       0.22646933, 0.9346783 , 0.30552356, 0.46986986, 0.71269641]

In [6]:
# But there is a more convenient way to load all the relevant information in an organized structure
from qube import qcodes_to_datafile, run_id_to_datafile

# Let's convert the data into a Datafile format with Dataset and Axis
df = run_id_to_datafile(run_id)

print(df) 
# As you can see, there are 2 datasets (y1 and y2) where each of them has an axis (x1).

Datafile
fullpath: None
datasets: 2 elements
[0] name: y1 - unit: nA - shape: (11, 5)
	axes: 3 elements
	[0] name: x1 - unit: V - shape: (11,) - dim: 0
	[1] name: counter_dim0 - unit: a.u. - shape: (11,) - dim: 0
	[2] name: counter_dim1 - unit: a.u. - shape: (5,) - dim: 1
[1] name: y2 - unit: pA - shape: (11, 5)
	axes: 3 elements
	[0] name: x1 - unit: V - shape: (11,) - dim: 0
	[1] name: counter_dim0 - unit: a.u. - shape: (11,) - dim: 0
	[2] name: counter_dim1 - unit: a.u. - shape: (5,) - dim: 1


## Types of sweep instructions
There are few pre-determined types:

- .sweep_linear(param, init, final, dim)
- .sweep_log(param, init, final, dim)
- .sweep_step(param, init, step_size, dim)
- .sweep_values(param, values, dim)
- .sweep_custom(param, f, dim) --> f is a function to generate sweep values with a given number of pts. values = f(pts)

## Multi-dimensional sweep

Imagine now we want to sweep x1 and x2 in different dimensions. 

We can simply add another instruction to our sweeper.

In [7]:
# We can actually reuse the sweeper instance created before.
sw.clear_instructions()

# we add sweep instructions
sw.sweep_linear(x1,0,-1,dim=1)
sw.sweep_linear(x2,0,+1,dim=2)

# Since it is the same sweeper, we don't need to define again the readouts, the waiting times, etc.
# we execute
sw.set_sweep_shape([11,5]) # [dim1 pts, dim2 pts, ...]
run_id = sw.execute()

Starting experimental run with id: 160. 
The measurement will take 0:00:03.302960


  0%|          | 0/55 [00:00<?, ?it/s]

In [8]:
df = run_id_to_datafile(run_id)
df
# Now, each dataset has 2 axes (x1 and x2) with their corresponding dimensions.

Datafile
fullpath: None
datasets: 2 elements
[0] name: y1 - unit: nA - shape: (11, 5)
	axes: 4 elements
	[0] name: x1 - unit: V - shape: (11,) - dim: 0
	[1] name: x2 - unit: V - shape: (5,) - dim: 1
	[2] name: counter_dim0 - unit: a.u. - shape: (11,) - dim: 0
	[3] name: counter_dim1 - unit: a.u. - shape: (5,) - dim: 1
[1] name: y2 - unit: pA - shape: (11, 5)
	axes: 4 elements
	[0] name: x1 - unit: V - shape: (11,) - dim: 0
	[1] name: x2 - unit: V - shape: (5,) - dim: 1
	[2] name: counter_dim0 - unit: a.u. - shape: (11,) - dim: 0
	[3] name: counter_dim1 - unit: a.u. - shape: (5,) - dim: 1

## Parallel sweep

Parallel sweep is simply to change two or more parameters at the same time.

Again, this is simple to define in the instructions for the sweep

In [9]:
sw.clear_instructions()
sw.sweep_linear(x1,0,-1,dim=1)
sw.sweep_linear(x2,0,+1,dim=1) # same dimension as x1
run_id = sw.execute(
            sweep_shape=[11,5], # we can pass it as argument in execute as well
)

Starting experimental run with id: 161. 
The measurement will take 0:00:03.387982


  0%|          | 0/55 [00:00<?, ?it/s]

In [10]:
df = run_id_to_datafile(run_id)
df
# x1 and x2 have indeed the same dimension

Datafile
fullpath: None
datasets: 2 elements
[0] name: y1 - unit: nA - shape: (11, 5)
	axes: 4 elements
	[0] name: x1 - unit: V - shape: (11,) - dim: 0
	[1] name: x2 - unit: V - shape: (11,) - dim: 0
	[2] name: counter_dim0 - unit: a.u. - shape: (11,) - dim: 0
	[3] name: counter_dim1 - unit: a.u. - shape: (5,) - dim: 1
[1] name: y2 - unit: pA - shape: (11, 5)
	axes: 4 elements
	[0] name: x1 - unit: V - shape: (11,) - dim: 0
	[1] name: x2 - unit: V - shape: (11,) - dim: 0
	[2] name: counter_dim0 - unit: a.u. - shape: (11,) - dim: 0
	[3] name: counter_dim1 - unit: a.u. - shape: (5,) - dim: 1

## Define initial and final configurations
Often, you want to define the initial conditions before starting a sweep. Similarly, after finish the measurement, you may want to return to an idle configuration (e.g. turn off the signal generator).

Sweeper allows you to set these configurations. Behind the screen, it will also save these "static" parameter values in the database.

In [38]:
# Let's create a dummy RF on/off parameter
rf = Parameter('RF_status', set_cmd=None, get_cmd=None, initial_value=0)

sw = Sweeper()
sw.set_readouts(y1, y2)
sw.pre_readout_wait = 0.1

# Let's initialize x1 to a custom value.
x1(0.5)

sw.sweep_linear(x1, 0, -1, dim=1)
sw.set_sweep_shape([11])

# Let's turn on the RF before the sweep.
sw.set_start_at({rf:1})

# We also set to turn off the RF after the sweep
sw.set_return_to({rf:0})

run_id = sw.execute()

Starting experimental run with id: 166. 
The measurement will take 0:00:01.181210


  0%|          | 0/11 [00:00<?, ?it/s]

## Let's explore the static configurations

We will use a SweeperContent instance which is the engine behind run_id_to_datafile.

In [44]:
from qube.measurement.content import SweeperContent
ds = load_by_id(run_id)
cont = SweeperContent(ds)

# we can access to the datasets from the last sweep
# cont.datasets

# more interestingly, we can access to the static configurations at the beginning and the end of the sweep
cont.statics

# as you can see, RF_status is 1 in "init" and 0 in "final"
# furthermore, note that x1 intiial value is 0.5 since we initialized it manually before the sweep.
# And the final value of x1 is, in this case, the last swept value = -1.0

{'init': [Static - name: x1 - unit: V - value: 0.5,
  Static - name: RF_status - unit: a.u. - value: 1.0],
 'final': [Static - name: x1 - unit: V - value: -1.0,
  Static - name: RF_status - unit: a.u. - value: 0.0]}

## Custom pre/post process

Pre- and post-processes refer to actions before and after the sweep. They are useful in situations where you want to generate and save a plot at the end of the experiment.

To show this capability, we will simply generate some printing functions.

In [48]:
sw = Sweeper()
sw.set_readouts(y1, y2)
sw.pre_readout_wait = 0.1
sw.add_pre_process(lambda: print('Starting sweep...'))
sw.add_pre_process(lambda: print('Second pre-process...')) # you can as many as you want

sw.add_post_process(lambda: print('End of sweep...'))
sw.add_post_process(lambda: print('Second post-process...'))

sw.set_sweep_shape([11]) # only repetitions
run_id = sw.execute()

Starting sweep...
Second pre-process...
Starting experimental run with id: 169. 
The measurement will take 0:00:01.207456


  0%|          | 0/11 [00:00<?, ?it/s]

End of sweep...
Second post-process...


## Custom pre/post readout processes

In contrary to pre/post-processes, the pre/post-readouts will be execute at every loop, meaning that if you sweep 10 points, these actions will be executed 10 times.

This is useful in situations where you want to, for example, prepare/reset an instrument before/after each readout.
In particular, you can trigger the AWG to generate only once the waveform or to clean the buffer of the oscilloscope.

In [51]:
sw = Sweeper()
sw.set_readouts(y1, y2)
sw.pre_readout_wait = 0.1

i = 0
def counter():
    global i
    i += 1
    
sw.add_pre_readout(lambda: print(f'Loop {i}'))
sw.add_pre_readout(lambda: print(f'Trigger AWG'))
sw.add_pre_readout(counter) 
sw.add_post_readout(lambda: print('Clean buffer after readout!'))

sw.set_sweep_shape([5]) # only repetitions
run_id = sw.execute()

Starting experimental run with id: 171. 
Loop 0
Trigger AWG
Clean buffer after readout!
Loop 1
Trigger AWG
Clean buffer after readout!
Loop 2
Trigger AWG
Clean buffer after readout!
Loop 3
Trigger AWG
Clean buffer after readout!
The measurement will take 0:00:00.544125


  0%|          | 0/5 [00:00<?, ?it/s]

Loop 4
Trigger AWG
Clean buffer after readout!


## Other examples of using Sweeper

In general, you can set a configuration to a Sweeper by indiviual methods such as set_sweep_shape, set_readouts, etc.
However, it is also possible to set all (or few) at once using set_config(...). Similarly, the same arguments can be passed to execute(...)

In [54]:
sw1 = Sweeper()
sw1.set_readouts(y1, y2)
sw1.pre_readout_wait = 0.1
sw1.set_sweep_shape([11])

# We can do the same thing with:
sw2 = Sweeper()
sw2.set_config(
    readouts=[y1, y2],
    pre_readout_wait=0.1,
    sweep_shape=[11],
)

In [55]:
# You can check all the accepted kwargs here:
help(sw2.set_config)

Help on method set_config in module qube.measurement.sweeper:

set_config(instructions: List[List] = None, sweep_shape: Iterable[int] = None, readouts: List[Union[qcodes.parameters.parameter.Parameter, qcodes.parameters.delegate_parameter.DelegateParameter]] = None, measurement: qcodes.dataset.measurements.Measurement = None, start_at: Dict[Union[qcodes.parameters.parameter.Parameter, qcodes.parameters.delegate_parameter.DelegateParameter], Any] = None, return_to: Dict[Union[qcodes.parameters.parameter.Parameter, qcodes.parameters.delegate_parameter.DelegateParameter], Any] = None, apply_method: Callable[[Dict[Union[qcodes.parameters.parameter.Parameter, qcodes.parameters.delegate_parameter.DelegateParameter], Any]], NoneType] = None, readout_method: Callable[[List[Union[qcodes.parameters.parameter.Parameter, qcodes.parameters.delegate_parameter.DelegateParameter]]], Dict[Union[qcodes.parameters.parameter.Parameter, qcodes.parameters.delegate_parameter.DelegateParameter], Any]] = None,

## Test run

You can also run a test without applying the sweep values. This is useful if you want to prepare few sweeps for running overnight.

In [57]:
# a dummy parameter with safety litims to check in the test run
x3 = Parameter('x3', unit='V', set_cmd=None, get_cmd=None, initial_value=0, vals=Numbers(-2, 2)) # with safety limits

sw = Sweeper()
sw.set_readouts(y1, y2)
sw.pre_readout_wait = 0.1
sw.sweep_linear(x3, -1, 1, dim=1) # we are inside the safety limits
sw.set_sweep_shape([11])

# We want to check the values + the entire sweep process without actually applying the sweep values.
sw.execute(test_run=True) # default it is False

Starting experimental run with id: 172. 
The measurement will take 0:00:01.211634


  0%|          | 0/11 [00:00<?, ?it/s]

172

In [62]:
# As you see, it executes the measurement, but it does not move x3. It does however readout the values.
# We can see this in the data
df = run_id_to_datafile(run_id)
df # no x3 axis

Datafile
fullpath: None
datasets: 2 elements
[0] name: y1 - unit: nA - shape: (5,)
	axes: 1 elements
	[0] name: counter_dim0 - unit: a.u. - shape: (5,) - dim: 0
[1] name: y2 - unit: pA - shape: (5,)
	axes: 1 elements
	[0] name: counter_dim0 - unit: a.u. - shape: (5,) - dim: 0

In [64]:
# We can check the safety limits validation
sw.clear_instructions()
sw.sweep_linear(x3, -5, 5, dim=1) # note that the values are out of the safety limits
sw.execute(
    sweep_shape=[11],
    test_run=True,
)
# it raises an error as soon as it finds a sweep value which is out of the safety limits.

ValueError: -5.0 is invalid: must be between -2 and 2 inclusive; Parameter: x3

## Sweep sequence

Extra information about the exact sequence when execute() is called can be found in the docstring.

In [66]:
help(Sweeper)

Help on class Sweeper in module qube.measurement.sweeper:

class Sweeper(builtins.object)
 |  Sweeper(name='Sweep')
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name='Sweep')
 |      Class for performing multi-dimensional sweep measurements with qcodes parameters.
 |      
 |      The user can define:
 |          - any number of sweep instructions for each parameter.
 |              Ex: .sweep_linear(V1, 0, 1, dim=1)
 |          - sweep shape: points for each dimension.
 |              Ex: .set_sweep_shape([2,3,4])  # 2, 3 and 4 pts for dim 1, 2 and 3 (respectively)
 |          - custom method to apply a swept value (default: param(value).
 |              Ex: .set_apply_method(f) where f takes a dictionary as argument.
 |              See docstring of .set_apply_method
 |          - custom method to readout (default: param())
 |              Ex: .set_readout_method(f) where f takes a list of qcodes parameters.
 |              See docstring of .set_readout_method
 |          -